In [9]:
import numpy as np
import pandas as pd
import pickle

# 1. Data
- Load data from pickle file
- split the data into independent and dependent
- split to train and test set

In [10]:
data = pickle.load(open('data_face_features_emotion.pickle',mode='rb'))

In [11]:
X = np.array(data['data']) # indendepent variable
y = np.array(data['label']) # dependent variable

In [12]:
X.shape , y.shape

((5326, 1, 128), (5326,))

In [13]:
X = X.reshape(-1,128)
X.shape

(5326, 128)

In [14]:
# split the data into train and test
from sklearn.model_selection import train_test_split

In [15]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [16]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4260, 128), (1066, 128), (4260,), (1066,))

# 2. Train Machine Learning

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

### Logistic Regression

In [18]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) # training logistic regression

C:\Users\Zele\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
def get_report(model, x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [20]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.31
Accuracy Test = 0.28
F1 Score Train = 0.17
F1 Score Test = 0.16


### Support Vector Machines

In [21]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [22]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.37
Accuracy Test = 0.27
F1 Score Train = 0.20
F1 Score Test = 0.14


### Random Forest

In [23]:
model_rf = RandomForestClassifier(n_estimators=10,)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [24]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 0.99
Accuracy Test = 0.25
F1 Score Train = 0.99
F1 Score Test = 0.33


## Voting Classifier

In [25]:
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [26]:
model_voting.fit(x_train,y_train)

C:\Users\Zele\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [27]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.73
Accuracy Test = 0.28
F1 Score Train = 0.60
F1 Score Test = 0.18


# 3. Parameter Tuning

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C':[3,5,7,10],
                             'svm__gamma':[0.1,0.3,0.5],
                             'rf__n_estimators':[5,10,20],
                             'rf__max_depth':[3,5,7],
                             'voting':['soft']
                         },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [30]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


C:\Users\Zele\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  17.7s


C:\Users\Zele\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  18.9s


...

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=  16.1s


C:\Users\Zele\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=  15.7s


C:\Users\Zele\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft']},
             scoring='accuracy', verbose=2)

In [31]:
model_best_estimator = model_grid.best_estimator_

In [32]:
model_grid.best_score_

0.3039906103286385

# 4. Save Model

In [34]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_emotion.pkl',mode='wb'))